In [1]:
import json
import re
from neo4j.v1 import GraphDatabase, basic_auth

In [2]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth = basic_auth("neo4j", "temppassword1"))
session = driver.session()

In [3]:
cia = []
with open("../factbook.json") as infile:
    for line in infile:
        cia_fact = json.loads(line)
        cia.append(cia_fact)

In [ ]:
for country in cia: #this will go through cia and plot everything
    b = country['region']
    if b == None:
        c = "None"
    else:
        b = b.replace(" ", "")
        c = re.sub('[^a-zA-Z0-9 \n\.]', '', b)
    name = country['countryname']
    code = country['countrycode']
    session.run("MERGE (a: "+c+" {name: {name}, code:{code}})", {"name": name, "code": code})

In [14]:
print (cia[10])

{'sections': {'Energy': [{'category_name': 'Electricity - production', 'section_title': 'Energy', 'category_data': '155 million kWh (2011 est.)'}, {'category_name': 'Electricity - consumption', 'section_title': 'Energy', 'category_data': '147.9 million kWh (2010 est.)'}, {'category_name': 'Electricity - exports', 'section_title': 'Energy', 'category_data': '0 kWh (2012 est.)'}, {'category_name': 'Electricity - imports', 'section_title': 'Energy', 'category_data': '0 kWh (2012 est.)'}, {'category_name': 'Electricity - installed generating capacity', 'section_title': 'Energy', 'category_data': '60,000 kW (2010 est.)'}, {'category_name': 'Electricity - from fossil fuels', 'section_title': 'Energy', 'category_data': '100% of total installed capacity (2010 est.)'}, {'category_name': 'Electricity - from nuclear fuels', 'section_title': 'Energy', 'category_data': '0% of total installed capacity (2010 est.)'}, {'category_name': 'Electricity - from hydroelectric plants', 'section_title': 'Energ